## **04. 잠재 요인 협업 필터링**

###- 확률적 경사 하강법을 이용한 확률 분해

In [59]:
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재요인 차원 K는 3 설정.
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN ],
              [np.NaN, 5, np.NaN, 3, 1 ],
              [np.NaN, np.NaN, 3, 4, 4 ],
              [5, 2, 1, 2, np.NaN ]])
num_users, num_items = R.shape
K=3

# P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 random한 값으로 입력합니다.
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [60]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)

    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]

    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

In [61]:
# SGD 기반으로 행렬 분해를 수행

# R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장.
non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]

steps=1000
learning_rate=0.01
r_lambda=0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트.
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값 구함
        eij = r - np.dot(P[i, :], Q[j, :].T)
        # Regularization을 반영한 SGD 업데이트 공식 적용
        P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0 :
        print("### iteration step : ", step," rmse : ", rmse)

### iteration step :  0  rmse :  3.2388050277987723
### iteration step :  50  rmse :  0.4876723101369648
### iteration step :  100  rmse :  0.1564340384819247
### iteration step :  150  rmse :  0.07455141311978046
### iteration step :  200  rmse :  0.04325226798579314
### iteration step :  250  rmse :  0.029248328780878973
### iteration step :  300  rmse :  0.022621116143829466
### iteration step :  350  rmse :  0.019493636196525135
### iteration step :  400  rmse :  0.018022719092132704
### iteration step :  450  rmse :  0.01731968595344266
### iteration step :  500  rmse :  0.016973657887570753
### iteration step :  550  rmse :  0.016796804595895633
### iteration step :  600  rmse :  0.01670132290188466
### iteration step :  650  rmse :  0.01664473691247669
### iteration step :  700  rmse :  0.016605910068210026
### iteration step :  750  rmse :  0.016574200475705
### iteration step :  800  rmse :  0.01654431582921597
### iteration step :  850  rmse :  0.01651375177473524
### iterati

In [62]:
#분해된 P와 Q함수를 P*Q.T로 예측 행렬을 만들어 출력

pred_matrix = np.dot(P, Q.T) # P @ Q.T 도 가능
print('\n예측 행렬:\n', np.round(pred_matrix, 3))


예측 행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]


## **08. 파이썬 추천 시스템 패키지-Surprise**

###- Surprise를 이용한 추천 시스템 구축

In [63]:
!pip install scikit-surprise

In [64]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [65]:
data= Dataset.load_builtin('ml-100k')
# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

In [66]:
!ls /root/.surprise_data/ml-100k

ml-100k


In [67]:
algo=SVD()
algo.fit(trainset)

In [68]:
predictions= algo.test(testset)
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5420490217434244, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=4.064088825693579, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.906510091281124, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.567075899332699, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.257356138581865, details={'was_impossible': False})]

In [69]:
[(pred.uid, pred.iid, pred.est)for pred in predictions[:3]]

[('120', '282', 3.5420490217434244),
 ('882', '291', 4.064088825693579),
 ('535', '507', 3.906510091281124)]

동일 패키지 내 다른 추천 예측 메서드인 predict()을 이용해 추천 예측

In [70]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함
uid=str(196)
iid= str(302)
pred=algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 3.99   {'was_impossible': False}


In [71]:
accuracy.rmse(predictions)

RMSE: 0.9509


0.9508991861025147

**[OS 파일 데이터를 Surprise 데이터 세트로 로딩]**




In [72]:
import pandas as pd

ratings = pd.read_csv('/content/ratings.csv')
ratings.to_csv('/content/ratings_noh.csv', index=False, header=False)

In [73]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file('/content/ratings_noh.csv', reader = reader)

In [74]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)

In [75]:
# 학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8708


0.8708344753692029

**[판다스 DataFrame에서 Surprise 데이터 세트로 로딩]**

In [76]:
print(ratings.columns)

Index(['1', '1.1', '4.0', '964982703'], dtype='object')


In [77]:
# 명시적으로 첫 번째 줄을 헤더로 처리
ratings = pd.read_csv('/content/ratings.csv', header=0)

# 데이터 내용 확인
print(ratings.head())
print(ratings.columns)

   1  1.1  4.0  964982703
0  1    3  4.0  964981247
1  1    6  4.0  964982224
2  1   47  5.0  964983815
3  1   50  5.0  964982931
4  1   70  3.0  964982400
Index(['1', '1.1', '4.0', '964982703'], dtype='object')


In [78]:
# 헤더 없이 데이터 로드
ratings = pd.read_csv('/content/ratings.csv', header=None)

# 열 이름 수동 설정
ratings.columns = ['userId', 'movieId', 'rating', 'timestamp']  # 필요한 열 이름에 맞게 변경

In [79]:
# 열 이름 재설정 후 저장
ratings.to_csv('/content/ratings_fixed.csv', index=False)

In [80]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('/content/ratings_fixed.csv')
# Print the actual column names to verify
print(ratings.columns)
# Output might be: Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

# Use the actual column names from the print statement:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
# Replace 'userId', 'movieId', 'rating' with actual names if they differ

trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')
RMSE: 0.8682


0.8681952927143516

###- 교차 검증과 하이퍼 파라미터 튜닝

In [81]:
from surprise.model_selection import cross_validate

ratings = pd.read_csv('/content/ratings_fixed.csv')
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv = 5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8707  0.8845  0.8686  0.8645  0.8736  0.8724  0.0067  
MAE (testset)     0.6698  0.6809  0.6662  0.6643  0.6727  0.6708  0.0058  
Fit time          2.34    2.43    1.82    1.78    1.84    2.04    0.28    
Test time         0.27    0.14    0.47    0.13    0.13    0.23    0.13    


{'test_rmse': array([0.87067389, 0.88447785, 0.8686039 , 0.86453146, 0.87360213]),
 'test_mae': array([0.66975423, 0.68087162, 0.66615616, 0.66429301, 0.67269149]),
 'fit_time': (2.3408896923065186,
  2.4308483600616455,
  1.8224124908447266,
  1.77769136428833,
  1.8425016403198242),
 'test_time': (0.27085304260253906,
  0.135847806930542,
  0.4744265079498291,
  0.13083171844482422,
  0.13369369506835938)}

In [82]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv = 3)
# 위의 ratings.csv 를 DataFrame으로 로딩한 데이터
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8766257301024302
{'n_epochs': 20, 'n_factors': 50}


### - Suprise를 이용한 개인화 영화 추천 시스템 구축

In [103]:
with open('/content/ratings_noh.csv', 'r') as f:
    for i in range(5):  # 첫 5줄만 확인
        print(f.readline())

1,3,4.0,964981247

1,6,4.0,964982224

1,47,5.0,964983815

1,50,5.0,964982931

1,70,3.0,964982400



In [105]:
ratings_noh = pd.read_csv('/content/ratings_noh.csv', header=None)  # 헤더가 없으므로 header=None
ratings_noh.columns = ['userId', 'movieId', 'rating', 'timestamp']  # 열 이름 설정


In [106]:
from surprise import Dataset, Reader

# Reader 설정
reader = Reader(rating_scale=(0.5, 5.0))

# 데이터프레임을 Surprise 데이터로 변환
data = Dataset.load_from_df(ratings_noh[['userId', 'movieId', 'rating']], reader)


In [107]:
ratings_noh.to_csv('/content/ratings_with_header.csv', index=False, header=True)


In [108]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
# ERROR
#algo.fit(data)

In [109]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5.0))
data_folds = DatasetAutoFolds(ratings_file='/content/ratings.csv', reader=reader)

trainset = data_folds.build_full_trainset()

In [110]:
algo = SVD(n_epochs=20, n_factors = 50, random_state=0)
algo.fit(trainset)

In [118]:
movies = pd.read_csv('/content/ratings_fixed.csv')
# userId = 9 의 movieId 데이터 추출하여 movieId = 42 데이터가 있는지 확인
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds == 42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId'] == 42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
       userId  movieId  rating   timestamp
34913     234       42     4.0  1004408338
55402     368       42     2.0   975828507
62318     414       42     2.0   961515844
85122     553       42     4.0  1219559052
90430     588       42     3.0   839316637
92168     597       42     3.0   941729264
92647     599       42     3.0  1498525483


In [119]:
print(ratings.columns)
print(movies.columns)


Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')
Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')


In [87]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [90]:
def get_unseen_surprise(ratings, movies, userId):
    # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()

    # 모든 영화들의 movieId를 리스트로 생성
    total_movies = movies['movieId'].tolist()

    # 모든 영화들의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
def get_unseen_surprise(ratings, movies, userId):
    # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()

    # 모든 영화들의 movieId를 리스트로 생성
    total_movies = movies['movieId'].tolist()

    # 모든 영화들의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:', len(seen_movies), '추천대상 영화수:', len(unseen_movies),
         '전체 영화수:', len(total_movies))
    return unseen_movies
unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 98855 전체 영화수: 100836


In [99]:
print(movies.columns)

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')


In [120]:
movies = pd.read_csv('/content/ratings.csv')  # 영화 제목 데이터 로드
# 기존 movies 데이터프레임에 제목 병합
movies = ratings(titles, on='movieId', how='left')
# movie_titles는 title 정보를 가진 데이터프레임


NameError: name 'titles' is not defined